## LLaMa3.2_Prompt_converting

In [13]:
import pandas as pd
import ast  # 문자열로 저장된 리스트를 리스트 객체로 변환

# 파일 경로
file_path = r"C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\QAData\학술지_all_final.csv"

# CSV 불러오기
df = pd.read_csv(file_path)

# "관련 문서" 열의 리스트 길이 구하기
def get_list_length(cell):
    try:
        # 만약 NaN이면 0 리턴
        if pd.isna(cell):
            return 0
        # 문자열로 저장된 리스트를 파싱해서 길이 반환
        parsed = ast.literal_eval(cell)
        return len(parsed) if isinstance(parsed, list) else 0
    except:
        return 0

# 각 행에 적용해서 길이 컬럼 추가
df["관련문서_길이"] = df["관련 문서"].apply(get_list_length)

# 최대 길이 찾기
max_length = df["관련문서_길이"].max()

print(f"관련 문서 최대 길이: {max_length}개")


관련 문서 최대 길이: 9개


In [15]:
import pandas as pd
import ast

# 파일 경로
file_path = r"C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\QAData\학술지_all_final.csv"

# CSV 불러오기
df = pd.read_csv(file_path)

def get_list_length(cell):
    try:
        if pd.isna(cell) or str(cell).strip() in ['', "''"]:  # 빈값 처리
            return 0
        parsed = ast.literal_eval(cell)
        # 리스트 형태이며, 내용이 ''이면 0으로 취급
        if isinstance(parsed, list):
            if all(str(x).strip() == '' for x in parsed):
                return 0
            return len(parsed)
        else:
            return 0
    except:
        return 0

def parse_list(cell):
    try:
        if pd.isna(cell) or str(cell).strip() in ['', "''"]:
            return []
        parsed = ast.literal_eval(cell)
        if isinstance(parsed, list):
            # 빈 문자열만 포함한 리스트면 빈 리스트로 처리
            if all(str(x).strip() == '' for x in parsed):
                return []
            return parsed
        else:
            return []
    except:
        return []

# 리스트 길이 컬럼 추가
df["관련문서_길이"] = df["관련 문서"].apply(get_list_length)

# 최대 길이 확인
max_length = df["관련문서_길이"].max()
print(f"관련 문서 최대 길이: {max_length}개\n")

# 최대 길이 행 필터링
longest_rows = df[df["관련문서_길이"] == max_length]

# 내용 출력
for idx, row in longest_rows.iterrows():
    print(f"질문: {row['질문']}\n")
    print(f"관련 문서 개수: {row['관련문서_길이']}")
    문서목록 = parse_list(row['관련 문서'])
    for i, 문장 in enumerate(문서목록, start=1):
        print(f"  ({i}) {문장}")
    print(f"\n답변: {row['답변']}")
    print("-" * 80)



관련 문서 최대 길이: 9개

질문: 매플슨 A 호흡회로(Magill circuit)에서 재호흡을 방지하기 위해 필요한 신선가스 유량은 어떻게 계산하며, 10kg 소아에서 권장되는 신선가스 유량은 얼마인가요?

관련 문서 개수: 9
  (1) 신선가스 유량 (L/min) = 175 x BW x 0.8 (Table 1)
  (2) Table 1. Guideline for fresh gas flow
  (3) | Body weight (kg) | Fresh gas flow (L/min) |
  (4) | --- | --- |
  (5) | 5 | 0.6 |
  (6) | 10 | 1.1 |
  (7) | 20 | 1.9 |
  (8) | 40 | 3.3 |
  (9) 위의 수치는 단순한 지침이므로 재호흡이 있는 경우에는 적절하게 유량을 조절하여야 한다.

답변: ##Reason: ##begin_quote## 신선가스 유량 (L/min) = 175 x BW x 0.8 (Table 1) ##end_quote## 라는 공식과 ##begin_quote## Table 1. Guideline for fresh gas flow ... | 10 | 1.1 | ##end_quote## 표에 따르면, 10kg 소아에서 권장되는 신선가스 유량은 1.1L/min입니다.
<ANSWER>: 매플슨 A 회로에서 10kg 소아의 권장 신선가스 유량은 1.1L/min입니다.
--------------------------------------------------------------------------------


In [4]:
import os
import json
import ast
import io

import pandas as pd
from tqdm import tqdm

# 1) 경로 설정
SRC_CSV   = r"C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\QAData\학술지_train_data_result(혜원).csv"
DST_JSONL = r"C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\QAData\학술지_LLaMa3.2_Fine-tuning_Dataset.jsonl"

# 2) CSV 인코딩 자동 탐색
def read_csv_flexible(path, encodings=("utf-8-sig", "cp949", "euc-kr", "utf-8")):
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc)
        except UnicodeDecodeError:
            continue
    # 전부 안될 시 -> 최종에는 바이너리로 읽어 오류 문자 대체
    with open(path, "rb") as f:
        raw = f.read()
    text = raw.decode(encodings[-1], errors="replace")
    return pd.read_csv(io.StringIO(text))

In [5]:
# 3) 문자열로 된 리스트 파싱 (JSON 또는 Python literal)
def safe_list_load(s: str):
    try:
        return json.loads(s)
    except json.JSONDecodeError:
        try:
            return ast.literal_eval(s)
        except Exception:
            return []

# 4) LLaMA-3 Instruction 포맷 생성
def build_llama3_instruction(question: str, ctx_list, cot_and_answer: str):
    # 질문 앞에 'Q. ' 접두
    q_text = f"Q. {question.strip()}"
    # 문맥 Bullet들(앞에 • 붙여서 개별 문맥 표시)
    bullets = "\n".join(f"• {c.strip()}" for c in ctx_list if c)
    # 조립
    return (
        "<s>[INST] "
        f"{q_text}\n\nContext:\n{bullets}"
        " [/INST] "
        f"{cot_and_answer.strip()}"
        " </s>"
    )

def main():
    # 5) CSV 로드
    df = read_csv_flexible(SRC_CSV)

    # 6) 컬럼 정리: strip + BOM 제거
    df.columns = [c.strip().lstrip("\ufeff") for c in df.columns]

    # 7) 한글→영문 매핑 (필요하다면)
    mapping = {
        "파일명":   "file",
        "질문":     "question",
        "관련 문서": "context",
        "답변":     "answer",
    }
    df = df.rename(columns={k: v for k, v in mapping.items() if k in df.columns})

    # 8) 필수 컬럼 체크
    for col in ("question", "context", "answer"):
        if col not in df.columns:
            raise ValueError(f"CSV에 '{col}' 컬럼이 없습니다.")

    # 9) JSONL으로 변환
    os.makedirs(os.path.dirname(DST_JSONL), exist_ok=True)
    with open(DST_JSONL, "w", encoding="utf-8") as fout:
        for _, row in tqdm(df.iterrows(), total=len(df)):
            q   = str(row["question"])
            ctx = safe_list_load(str(row["context"]))
            cot = str(row["answer"])  # 이미 CoT 형식으로 ##Reason…<ANSWER>:… 를 포함

            llama_text = build_llama3_instruction(q, ctx, cot)
            fout.write(json.dumps({"text": llama_text}, ensure_ascii=False) + "\n")

    print(f"Saved LLaMA-3 JSONL dataset to {DST_JSONL}")

if __name__ == "__main__":
    main()

100%|██████████| 900/900 [00:01<00:00, 620.64it/s]

Saved LLaMA-3 JSONL dataset to C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\QAData\학술지_LLaMa3.2_Fine-tuning_Dataset.jsonl


## LLaMa3.2_Fine-tuning code

In [6]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig
from trl import SFTTrainer
import huggingface_hub
from dotenv import load_dotenv

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [7]:
# 0) 환경 변수 & HF 로그인 (허깅페이스 API TOKEN 발급) 
load_dotenv()
huggingface_hub.login(os.getenv("HF_TOKEN"))

# 1) 설정 
base_model     = "meta-llama/Llama-3.2-1B" # 베이스 모델
dataset_path   = r"C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\QAData\학술지_LLaMa3.2_Fine-tuning_Dataset.jsonl" # Custom Dataset
new_model_name = "llama3.2-1b-boaz-medical" # 출력 모델 이름 설정

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [8]:
# 2) JSONL 로드(우리는 전체를 train set으로 쓰고, shuffle해서 데이터 샘플 간 순서 섞기)
ds = load_dataset(
    "json",
    data_files={"train": dataset_path},
    split="train"
).shuffle()

Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
# 3) QLoRA용 4-bit 양자화 설정
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,               # 4bit로 모델 가중치를 로드하여 메모리 사용량 크게 절감
    bnb_4bit_quant_type="nf4",       # NF4(NormalFloat4) 양자화 방식 사용
    bnb_4bit_compute_dtype=torch.float16,  # 내부 계산 시 float16 포맷 사용
    bnb_4bit_use_double_quant=False, # 이중 양자화(double quantization)는 미적용
)

In [12]:
# 4) 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    base_model,                      # 사전학습된 LLaMA-3.2 1B 모델 식별자
    trust_remote_code=True,          # 모델 저장소의 custom code(예: Llama-3 사용자 정의)를 허용
    quantization_config=quant_config,# 위에서 생성한 4bit 양자화 설정 적용
    device_map="auto",               # 사용 가능한 GPU/CPU 장치에 자동으로 분산 배치
)

# LoRA fine-tuning 시 반드시 끄기
model.config.use_cache = False       # generation시 사용하는 past_key/value 캐시 비활성화
model.config.pretraining_tp = 1      # tensor parallelism을 1로 고정(단일 GPU 모드)

ValueError: `rope_scaling` must be a dictionary with with two fields, `type` and `factor`, got {'factor': 32.0, 'high_freq_factor': 4.0, 'low_freq_factor': 1.0, 'original_max_position_embeddings': 8192, 'rope_type': 'llama3'}

In [ ]:
# 5) 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    base_model,                      # 동일한 base_model에서 토크나이저 불러오기
    trust_remote_code=True,          # custom tokenizer 코드 허용
)

tokenizer.pad_token    = tokenizer.eos_token  # 패딩 토큰을 eos_token과 동일하게 설정
tokenizer.padding_side = "right"              # 오른쪽에 패딩 추가

In [ ]:
# 6) LoRA(PEFT) 설정
peft_config = LoraConfig(
    task_type="CAUSAL_LM",   # LoRA 적용 - 답변 생성 Task
    inference_mode=False,    # 학습 모(LoRA 가중치 학습 허용)
    r=32,                    # LoRA low-rank 차원(rank) -> 작을수록 메모리·연산량 감소
    lora_alpha=16,           # LoRA 스케일링 계수(alpha) -> 작을수록 보수적으로 학습
    lora_dropout=0.1,        # 10% 드롭아웃으로 과적합 방지
    bias="none",             # LoRA 어댑터에는 편향(bias) 레이어 비활성화(편향까지 추가하면 불안정해질 수 있음)
)

In [ ]:
# 7) 학습 인자
training_args = TrainingArguments(
    output_dir="./results_llama3_1b",     # 체크포인트와 로그 저장 디렉터리
    per_device_train_batch_size=1,        # GPU 당 배치 사이즈, 실제 GPU 하나당 매 스텝에 1개 샘플을 로드
    gradient_accumulation_steps=8,        # 매 8 스텝마다 한 번씩 역전파를 수행해 효과적으로 배치 크기를 8로 늘린 것과 같음
    num_train_epochs=3,                   # 전체 데이터를 3회 반복 학습
    learning_rate=2e-4,                   # 학습률 0.0002
    optim="paged_adamw_32bit",            # 32-bit AdamW 옵티마이저 (사용 빈도가 낮은 파라미터 페이지(메모리 블록)를 자동으로 CPU 메모리나 디스크로 옮겼다가, 필요할 때만 GPU로 불러오는 방식)
    fp16=True,                            # FP16 mixed-precision 활성화
    bf16=False,                           # BF16 사용 안 함
    save_steps=50,                        # 50 스텝마다 모델 저장
    logging_steps=50,                     # 50 스텝마다 로깅
    weight_decay=0.01,                    # 가중치 감쇠 0.01, L2계수
    warmup_ratio=0.03,                    # 전체의 3%만큼 워밍업 단계, 워밍업 단계에 learning rate를 선형 증가시켜 안정적인 학습을 도움
    max_grad_norm=0.3,                    # 그래디언트 클리핑 최대 L2 노름 0.3
    lr_scheduler_type="constant",         # constant : 학습 내내 고정된 학습률을 사용
    report_to="tensorboard",              # 텐서보드로 로그 전송
    group_by_length=True,                 # 길이가 비슷한 시퀀스끼리 묶어서 배치로 구성, 패딩 최소화
)

In [ ]:
# 8) SFTTrainer 생성
trainer = SFTTrainer(
    model=model,                       # fine-tune할 모델
    train_dataset=ds,                  # 학습 데이터셋
    peft_config=peft_config,           # LoRA 설정
    dataset_text_field="text",         # 데이터셋 텍스트 필드 이름
    tokenizer=tokenizer,               # 토크나이저
    args=training_args,                # 학습 인자
    packing=False,                     # packing 비활성화(토크나이저 별도, 문장별 경계를 명확히 유지해야 할 때 주로 사용)
)

In [ ]:
# 9) 학습 & 저장
trainer.train()                       # Fine-tuning 시작
trainer.save_model(new_model_name)    # 학습된 LoRA 어댑터 + Config 저장
print("✅ Fine-tuning 완료. Saved to:", new_model_name)